In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b3.dev12
cdv, version 1.1.2
Python 3.10.6


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000

In [3]:
# from setup-sim notebook
# alice 4070016934
alice_sk = PrivateKey.from_bytes(bytes.fromhex("67959bf0ade73bc2aa69cab59d0aa1fded890696f178393f7740b330a808f7c6"))
alice_pk = G1Element.from_bytes(bytes.fromhex("96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])

In [4]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash,
    alice_pk
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch1dmjr50rp03dpa330lml3kczlk5lveqd63nx5qf82l7d7l3fuv9zsdq4rrm'

# Create Alice's Waiting Room Coin

In [5]:
%%bash
chia wallet send -f 4070016934 -t txch1dmjr50rp03dpa330lml3kczlk5lveqd63nx5qf82l7d7l3fuv9zsdq4rrm -a 1.000000000001

Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '38401aad891c6ecd90a91150989b51e9c86d844e07387ed77338beda85b199e7', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 4070016934 -tx 0xc94e3e5691bb30854468a232139bc3316c98a164276d1c0bfa5ecdc9d4e12669' to get status


In [6]:
%%bash
cdv decode txch1dmjr50rp03dpa330lml3kczlk5lveqd63nx5qf82l7d7l3fuv9zsdq4rrm

6ee43a3c617c5a1ec62ffeff1b605fb53ecc81ba8ccd4024eaff9befc53c6145


In [7]:
%%bash
cdv rpc coinrecords --by puzzlehash "6ee43a3c617c5a1ec62ffeff1b605fb53ecc81ba8ccd4024eaff9befc53c6145"

[
    {
        "coin": {
            "amount": 1000000000001,
            "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
            "puzzle_hash": "0x6ee43a3c617c5a1ec62ffeff1b605fb53ecc81ba8ccd4024eaff9befc53c6145"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1664467253
    }
]


## Clawback

In [8]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = "mainnet"
genesis_challenge = config["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
genesis_challenge

'ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb'

In [9]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_puzzle_coin = coin_records[0].coin
alice_waiting_room_puzzle_coin

Coin { parent_coin_info: df2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b, puzzle_hash: 6ee43a3c617c5a1ec62ffeff1b605fb53ecc81ba8ccd4024eaff9befc53c6145, amount: 1000000000001 }

In [10]:
from utils import print_json
from chia.types.coin_spend import CoinSpend

#  (defconstant CLAWBACK -1)
solution = Program.to([
    bytes.fromhex('900ddeed'), # launcher_coin_announcement
    -1
])

coin_spend = CoinSpend(
        alice_waiting_room_puzzle_coin,
        alice_waiting_room_puzzle,
        solution
)
print_json(coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000001,
        "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
        "puzzle_hash": "0x6ee43a3c617c5a1ec62ffeff1b605fb53ecc81ba8ccd4024eaff9befc53c6145"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff09ff12ff2f80ffff01ff04ffff04ff14ffff01ff648080ffff04ffff04ff16ffff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff0bffff01ff9072657475726e20746f2073656e646572808080ff80808080ffff01ff02ffff03ffff09ff1aff2f80ffff01ff04ffff04ff16ffff04ff3effff01ff8601d1a94a2001808080ffff04ffff04ff18ffff04ff17ff808080ffff04ffff04ff2effff04ff17ff808080ff80808080ffff01ff04ffff04ff1cffff04ff17ff808080ff808080ff018080ff0180ffff04ffff01ffffff323dff523fffff81ff01ff33ff3ea0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff018080ffff04ffff01a0d210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387ffff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd7

In [11]:
from chia.types.spend_bundle import SpendBundle


message: bytes = bytes("return to sender", "utf-8")
sig: G2Element = AugSchemeMPL.sign(
    alice_sk,
    message
    + alice_waiting_room_puzzle_coin.name()
    #+ bytes.fromhex(genesis_challenge),
    + bytes.fromhex("ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb")
)

spend_bundle = SpendBundle([coin_spend], sig)
print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0x96e063d6508437059da8a062fe0c7ee03820633d0a62a6c27f4d6a18078394dc0a604f6e340ceb865ad8273a32608eae06d96d6aadeec3bb63ada4b6ebb6ce198115394a7461a54d1829a4951d703cbf4203f3704238bf3e4c31c1ca00956ee2",
    "coin_spends": [
        {
            "coin": {
                "amount": 1000000000001,
                "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
                "puzzle_hash": "0x6ee43a3c617c5a1ec62ffeff1b605fb53ecc81ba8ccd4024eaff9befc53c6145"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff09ff12ff2f80ffff01ff04ffff04ff14ffff01ff648080ffff04ffff04ff16ffff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff0bffff01ff9072657475726e20746f2073656e646572808080ff80808080ffff01ff02ffff03ffff09ff1aff2f80ffff01ff04ffff04ff16ffff04ff3effff01ff8601d1a94a2001808080ffff04ffff04ff18ffff04ff17ff808080ffff04ffff04ff2effff04ff17ff808080ff80808080ffff01ff04ffff04ff1cffff04ff17ff808080f

## Clawback is not available yet

In [12]:
%%bash
cdv sim -n tic-tac-toe farm --blocks 10
chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height"

Farmed 10 Transaction blocks
Block Height is now: 16
16


In [13]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

{'status': 'PENDING', 'success': True}


## Clawback is available

In [12]:
%%bash
cdv sim -n tic-tac-toe farm --blocks 100
chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height"

Farmed 100 Transaction blocks
Block Height is now: 239
239


In [13]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

{'status': 'SUCCESS', 'success': True}


In [17]:
# farm
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

coin_records[0]

CoinRecord(coin=Coin { parent_coin_info: df2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b, puzzle_hash: 6ee43a3c617c5a1ec62ffeff1b605fb53ecc81ba8ccd4024eaff9befc53c6145, amount: 1000000000001 }, confirmed_block_index=6, spent_block_index=240, coinbase=False, timestamp=1664467253)